<a href="https://colab.research.google.com/github/helen-mark/phonecall/blob/master/MCP_Call_Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get install zstd
!curl -fsSL https://ollama.com/install.sh | sh
!pip install ollama

!pip list | grep ollama

!ollama list

import subprocess, time
print("Запускаю Ollama сервер...")
subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(10)

import requests
try:
    response = requests.get("http://localhost:11434/api/tags", timeout=5)
    if response.status_code == 200:
        print("✅ Ollama сервер работает")
    else:
        print(f"⚠️  Ollama сервер отвечает с кодом {response.status_code}")
except:
    print("❌ Ollama сервер не отвечает")

print("Скачиваю модель mistral-nemo:12b...")
!ollama pull mistral-nemo:12b
print('✅ Done')



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
zstd is already the newest version (1.4.8+dfsg-3build1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading ollama-linux-amd64.tar.zst
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
ollama                                   0.6.1
Error: ollama server not responding - could not connect to ollama server, run 'ollama serve' to start it
Запускаю Ollama сервер...


In [2]:
CONFIG = {
    # GitHub
    "github_repo": "https://github.com/helen-mark/phonecall.git",
    "repo_name": 'phonecall',

    # Google Drive пути
    "drive_root": "/content/drive/MyDrive",
    "project_folder": "MCP_Call_Analytics",

    # Папки для данных
    "folders": {
        "audio_raw": "audio_raw",
        "audio_processed": "audio_processed",
        "csv_calls": "csv_calls",
        "saved_results": "saved_results",
        "logs": "logs"
    },

    # Модели
    "llm_model": "mistral-nemo:12b",

    # Настройки обработки
    "check_interval": 30,  # секунды между проверками новых файлов
    "auto_process": True   # Автоматически обрабатывать новые файлы
}

In [3]:
import os
# ====================
# 🚀 ЗАГРУЗКА КОДА ИЗ GITHUB РЕПОЗИТОРИЯ
# ====================

from google.colab import files
import sys
from google.colab import drive

print("📦 Загружаю проект из GitHub...")

repo_url = CONFIG["github_repo"]
repo_name = CONFIG["repo_name"]

!rm -rf {repo_name} 2>/dev/null || true

!git clone {repo_url}

%cd {repo_name+ '/colab'}

print("\n📁 Project structure:")
!find . -name "*.py" | head -20

project_path = os.getcwd()
sys.path.insert(0, project_path)
print(f"\n📍 Проект загружен в: {project_path}")
print(f"📦 Python path обновлен")

print("\n2️⃣ Монтирую Google Drive...")
drive.mount('/content/drive', force_remount=True)

base_path = os.path.join(CONFIG['drive_root'], CONFIG['project_folder'])

for folder_name, folder_path in CONFIG['folders'].items():
    full_path = os.path.join(base_path, folder_path)
    os.makedirs(full_path, exist_ok=True)
    print(f"   ✅ {folder_name}: {full_path}")

required_files = ["main.py", "interactive.py", "mcp_orchestrator.py"]
missing_files = []

for file in required_files:
    if os.path.exists(file):
        print(f"✅ Найден: {file}")
    else:
        missing_files.append(file)
        print(f"❌ Отсутствует: {file}")

if missing_files:
    print(f"\n⚠️  Отсутствуют файлы: {missing_files}")
    print("Проверьте структуру репозитория на GitHub")
else:
    print("\n🎉 Все файлы загружены успешно!")

# print("📤 ЗАГРУЗИТЕ CSV ФАЙЛЫ ЗВОНКОВ:")
# csv_files = files.upload()

# for filename in csv_files.keys():
#     if filename.endswith('.csv'):
#         os.rename(filename, f"/content/csv_calls/{filename}")
#         print(f"✅ CSV файл: {filename}")



📦 Загружаю проект из GitHub...
Cloning into 'phonecall'...
remote: Enumerating objects: 100, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (75/75), done.
Receiving objects: 100% (100/100), 301.48 KiB | 1.79 MiB/s, done.
remote: Total 100 (delta 41), reused 76 (delta 23), pack-reused 0 (from 0)
Resolving deltas: 100% (41/41), done.
/content/phonecall/colab

📁 Project structure:
./main.py
./colab_drive_setup.py
./interactive.py
./mcp_orchestrator.py
./colab_minimal.py

📍 Проект загружен в: /content/phonecall/colab
📦 Python path обновлен

2️⃣ Монтирую Google Drive...
Mounted at /content/drive
   ✅ audio_raw: /content/drive/MyDrive/MCP_Call_Analytics/audio_raw
   ✅ audio_processed: /content/drive/MyDrive/MCP_Call_Analytics/audio_processed
   ✅ csv_calls: /content/drive/MyDrive/MCP_Call_Analytics/csv_calls
   ✅ saved_results: /content/drive/MyDrive/MCP_Call_Analytics/saved_results
   ✅ logs: /content/drive/MyDrive/MCP_Call_Analytics/logs
✅ Найден: m

In [4]:
!pip install pydub

In [5]:
print("📦 Устанавливаю OpenAI Whisper...")
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# 2. Проверьте установку
import whisper
print(f"✅ Whisper установлен, версия: {whisper.__version__}")

# 3. Скачайте модель (выберите нужный размер)
MODEL_SIZE = "large"  # или "small", "medium", "large", "large-v3"
# print(f"📥 Скачиваю модель {MODEL_SIZE}...")
# model = whisper.load_model(MODEL_SIZE)
# print("✅ Модель загружена")

📦 Устанавливаю OpenAI Whisper...
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
✅ Whisper установлен, версия: 20250625


In [6]:
import json
import pandas as pd

"""Явное указание пути"""
import sys

# Явно добавляем путь к вашему репозиторию
phonecall_path = "/content/phonecall"  # ИЗМЕНИТЕ НА ВАШ ПУТЬ
if phonecall_path not in sys.path:
    sys.path.insert(0, phonecall_path)

from phonecall.preprocess_calls_full import AudioProcessor


class AudioFileMonitor:
    """Мониторит новые аудиофайлы в папке audio_raw"""

    def __init__(self, audio_raw_path, csv_path, log_path):
        self.audio_raw_path = audio_raw_path
        self.csv_path = csv_path
        self.log_path = log_path
        self.processed_files = self._load_processed_files()

        os.makedirs(log_path, exist_ok=True)
        self.log_file = os.path.join(log_path, "audio_processing.log")

    def _load_processed_files(self):
        """Загружает список уже обработанных файлов из CSV"""
        processed = set()

        # Проверяем CSV файлы
        if os.path.exists(self.csv_path):
            csv_files = [f for f in os.listdir(self.csv_path) if f.endswith('.csv')]
            for csv_file in csv_files:
                try:
                    df = pd.read_csv(os.path.join(self.csv_path, csv_file))
                    if 'source_file' in df.columns:
                        processed.update(df['source_file'].dropna().unique())
                except:
                    pass

        # # Также проверяем логи
        # log_file = os.path.join(self.log_path, "processed_audio.json")
        # if os.path.exists(log_file):
        #     with open(log_file, 'r') as f:
        #         processed.update(json.load(f))

        return processed

    def check_new_files(self):
        """Проверяет наличие новых аудиофайлов"""
        if not os.path.exists(self.audio_raw_path):
            return []

        # Получаем все аудиофайлы
        audio_extensions = ['.mp3', '.wav', '.m4a', '.flac', '.ogg']
        all_files = []

        for ext in audio_extensions:
            all_files.extend(Path(self.audio_raw_path).glob(f"*{ext}"))
            all_files.extend(Path(self.audio_raw_path).glob(f"*{ext.upper()}"))

        # Фильтруем новые файлы
        new_files = []
        for file_path in all_files:
            file_name = file_path.name
            if file_name not in self.processed_files:
                new_files.append(str(file_path))

        return new_files

    def log_processing(self, audio_file, status, details=""):
        """Логирует процесс обработки"""
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log_entry = f"{timestamp} | {audio_file} | {status} | {details}\n"

        # Записываем в лог-файл
        with open(self.log_file, 'a', encoding='utf-8') as f:
            f.write(log_entry)

        # Обновляем список обработанных файлов
        if status == "COMPLETED":
            self.processed_files.add(os.path.basename(audio_file))
            self._save_processed_files()

        print(f"📝 {log_entry.strip()}")

    def _save_processed_files(self):
        """Сохраняет список обработанных файлов"""
        log_file = os.path.join(self.log_path, "processed_audio.json")
        with open(log_file, 'w') as f:
            json.dump(list(self.processed_files), f)

# Инициализируем монитор
AUDIO_RAW_PATH = os.path.join(base_path, CONFIG['folders']['audio_raw'])
CSV_PATH = os.path.join(base_path, CONFIG['folders']['csv_calls'])
LOG_PATH = os.path.join(base_path, CONFIG['folders']['logs'])

#monitor = AudioFileMonitor(AUDIO_RAW_PATH, CSV_PATH, LOG_PATH)

/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


In [8]:
import os
import json
import time
import shutil
import pandas as pd
from datetime import datetime
from pathlib import Path
import subprocess
from phonecall.preprocess_calls_full import AudioProcessor
from phonecall.assign_tags_from_fixed_list import JsonFileTaggingAgent


class SmartAudioProcessor:
    """Полная реализация инкрементальной обработки с батчами"""

    def __init__(self, drive_audio_path, output_csv_path,
                 total_space_gb=80, batch_size_gb=2):
        """
        Args:
            drive_audio_path: путь к аудиофайлам в Google Drive
            output_csv_path: путь для сохранения CSV результатов
            total_space_gb: общее место в Colab (по умолчанию 80GB)
            batch_size_gb: размер одного батча для обработки (по умолчанию 2GB)
        """
        my_tags = [
          "низкое_качество_стирки_или_чистки",
          "не_заменили_ковры_вовремя",
          "клиент_хочет_добавить_ковры",
          "клиент_хочет_меньше_ковров",
          "погашение_долга",
          "расторжение_договора",
          "возобновление_услуг",
          "долго_нет_ответа_на_заявку",
          "лишняя_доставка",
          "доставили_не_те_ковры",
          "не_выставлен_вовремя_счет",
          "неверная_сумма_в_счете",
          "ковер_забрали_без_причины",
          "забрали_не_тот_ковер",
          "менеджер_нагрубил_клиенту",
          "неоправданно_высокие_цены",
          "неоправданный_рост_цен",
          "новый_клиент_заключение_договора",
          "консультация_или_уточнение_деталей",
          "поменять_спецификации",
          "менеджер_обещал_но_не_связался_с_клиентом",
          "клиент_уходит_к_конкурентам",
          "приостановить_услуги",
          "ошибка_в_документах"
          ]
        self.drive_audio_path = drive_audio_path
        self.output_csv_path = output_csv_path
        self.total_space = total_space_gb
        self.batch_size = batch_size_gb
        self.ap = AudioProcessor(model_size='large')
        self.tagger = JsonFileTaggingAgent(
          model_name="mistral-nemo:12b",
          tags_list=my_tags
        )

        # Локальные рабочие директории
        self.local_temp_dir = "/content/temp_audio"
        self.local_whisper_dir = "/content/whisper_output"
        self.local_batch_dir = "/content/current_batch"

        # Создаем локальные директории
        for dir_path in [self.local_temp_dir, self.local_whisper_dir, self.local_batch_dir]:
            os.makedirs(dir_path, exist_ok=True)

        # Файл для логирования обработанных файлов
        self.processed_files_log = "/content/drive/MyDrive/MCP_Call_Analytics/processed_files.json"

        # CSV для промежуточных результатов
        self.partial_results_csv = self.output_csv_path  # "/content/partial_results.csv"

        print(f"🎯 Инициализация SmartAudioProcessor")
        print(f"📁 Аудиофайлы: {drive_audio_path}")
        print(f"💾 Лимит батча: {batch_size_gb} GB")
        print(f"📊 Общее место: {total_space_gb} GB")

    # ========== ОСНОВНОЙ МЕТОД ==========

    def process_large_dataset(self):
        """Основной метод: обрабатывает весь датасет батчами"""

        print("="*60)
        print("🚀 НАЧАЛО ОБРАБОТКИ БОЛЬШОГО ДАТАСЕТА")
        print("="*60)

        # 1. Получаем ВСЕ файлы (только имена, не скачивая)
        all_files = self._list_files_without_download()
        if not all_files:
            print("❌ Нет файлов для обработки")
            return []

        total_files = len(all_files)
        total_gb = self._estimate_total_size_gb(all_files)

        print(f"📊 Всего файлов: {total_files:,}")
        print(f"📊 Общий объем: {total_gb:.1f} GB")
        print(f"🎯 Размер батча: {self.batch_size} GB")
        print(f"🎯 Примерное количество батчей: {int(total_gb / self.batch_size) + 1}")

        # 2. Определяем, какие уже обработаны
        processed = self._load_processed_list()
        to_process = [f for f in all_files if f not in processed]

        print(f"\n📈 СТАТУС ОБРАБОТКИ:")
        print(f"✅ Уже обработано: {len(processed):,} файлов")
        print(f"🎯 Осталось обработать: {len(to_process):,} файлов")
        print(f"📊 Прогресс: {len(processed)/total_files*100:.1f}%")

        if not to_process:
            print("\n🎉 Все файлы уже обработаны!")
            return []


        # 3. Группируем файлы по батчам
        batches = self._create_batches(to_process)
        print(f"\n📦 Создано {len(batches)} батчей для обработки")

        # 4. Обрабатываем каждый батч
        all_results = []
        start_time = time.time()

        for batch_num, batch_files in enumerate(batches, 1):
            batch_start = time.time()

            print(f"\n{'='*50}")
            print(f"⚡ БАТЧ {batch_num}/{len(batches)}")
            print(f"{'='*50}")

            batch_size_gb = sum(self._get_file_size_gb(f) for f in batch_files)
            print(f"📁 Файлов в батче: {len(batch_files)}")
            print(f"💾 Размер батча: {batch_size_gb:.2f} GB")

            # Обрабатываем батч
            try:
                batch_results = self._process_batch(batch_files, batch_num)

                if batch_results:
                    all_results.extend(batch_results)

                    # Сразу сохраняем результаты
                    self._save_batch_results(batch_results, batch_num)

                    # Обновляем список обработанных
                    self._update_processed_list(batch_files)

                # Расчет времени
                batch_time = time.time() - batch_start
                avg_time_per_file = batch_time / len(batch_files) if batch_files else 0

                print(f"\n✅ Батч {batch_num} завершен за {batch_time:.1f} сек")
                print(f"⏱️  Среднее время на файл: {avg_time_per_file:.1f} сек")

                # Прогноз оставшегося времени
                remaining_batches = len(batches) - batch_num
                if remaining_batches > 0:
                    estimated_remaining = remaining_batches * (time.time() - start_time) / batch_num
                    print(f"📅 Осталось времени: ~{estimated_remaining/60:.1f} мин")

            except Exception as e:
                print(f"❌ Ошибка обработки батча {batch_num}: {e}")
                import traceback
                traceback.print_exc()

                # Пытаемся продолжить со следующим батчем
                continue

        # 5. Финальная консолидация результатов
        if all_results:
            self._consolidate_results()

            total_time = time.time() - start_time
            print(f"\n{'='*60}")
            print(f"🎉 ОБРАБОТКА ЗАВЕРШЕНА!")
            print(f"{'='*60}")
            print(f"📊 Обработано файлов: {len(all_results):,}")
            print(f"⏱️  Общее время: {total_time/60:.1f} мин")
            print(f"⚡ Скорость: {len(all_results)/total_time*60:.1f} файлов/мин")
            print(f"💾 Результаты сохранены в: {self.output_csv_path}")

        return all_results

    # ========== НЕДОСТАЮЩИЕ МЕТОДЫ ==========

    def _list_files_without_download(self):
        """Получает список всех аудиофайлов без их скачивания"""
        if not os.path.exists(self.drive_audio_path):
            print(f"❌ Папка не найдена: {self.drive_audio_path}")
            return []

        # Поддерживаемые аудиоформаты
        audio_extensions = ['.mp3', '.wav', '.m4a', '.flac', '.ogg', '.aac', '.wma']

        all_files = []
        for ext in audio_extensions:
            # Ищем файлы с расширением (регистронезависимо)
            pattern = f"*{ext}"
            files = list(Path(self.drive_audio_path).glob(pattern.lower()))
            files.extend(Path(self.drive_audio_path).glob(pattern.upper()))

            for file_path in files:
                if file_path.is_file():
                    all_files.append(str(file_path))

        # Сортируем по имени для воспроизводимости
        all_files.sort()

        print(f"🔍 Найдено {len(all_files)} аудиофайлов")
        if all_files:
            print(f"📅 Диапазон дат: {self._get_file_dates_range(all_files[:5])}")

        return all_files

    def _get_file_dates_range(self, file_paths):
        """Возвращает диапазон дат создания файлов"""
        if not file_paths:
            return "Нет файлов"

        dates = []
        for file_path in file_paths[:5]:  # Берем первые 5 для примера
            try:
                mod_time = os.path.getmtime(file_path)
                dates.append(datetime.fromtimestamp(mod_time))
            except:
                pass

        if not dates:
            return "Даты недоступны"

        min_date = min(dates).strftime("%d.%m.%Y")
        max_date = max(dates).strftime("%d.%m.%Y")

        return f"{min_date} - {max_date}"

    def _estimate_total_size_gb(self, file_paths):
        """Оценивает общий размер файлов в GB"""
        total_bytes = 0

        for file_path in file_paths:
            try:
                total_bytes += os.path.getsize(file_path)
            except:
                # Если не можем получить размер, предполагаем средний
                total_bytes += 50 * 1024 * 1024  # 50 MB

        return total_bytes / (1024**3)  # Конвертируем в GB

    def _get_file_size_gb(self, file_path):
        """Возвращает размер файла в GB"""
        try:
            return os.path.getsize(file_path) / (1024**3)
        except:
            return 0.05  # Предполагаем 50 MB если не можем получить размер

    def _load_processed_list(self):
        """Загружает список уже обработанных файлов"""
        processed_files = set()

        # 1. Проверяем файл с логами обработанных
        # if os.path.exists(self.processed_files_log):
        #     try:
        #         with open(self.processed_files_log, 'r') as f:
        #             data = json.load(f)
        #             if isinstance(data, list):
        #                 processed_files.update(data)
        #             print(f"📖 Загружен лог обработанных файлов: {len(processed_files)} записей")
        #     except Exception as e:
        #         print(f"⚠️  Ошибка чтения лога: {e}")

        # 2. Проверяем CSV файл с результатами
        if os.path.exists(self.output_csv_path):
            try:
                df = pd.read_csv(self.output_csv_path)
                if 'source_audio' in df.columns:
                    csv_processed = set(df['source_audio'].dropna().unique())
                    processed_files.update(csv_processed)
                    print(f"📊 Из CSV загружено: {len(csv_processed)} файлов")
            except Exception as e:
                print(f"⚠️  Ошибка чтения CSV: {e}")

        # 3. Проверяем частичные результаты
        if os.path.exists(self.partial_results_csv):
            try:
                df_partial = pd.read_csv(self.partial_results_csv)
                if 'source_audio' in df_partial.columns:
                    partial_processed = set(df_partial['source_audio'].dropna().unique())
                    processed_files.update(partial_processed)
                    print(f"📄 Из частичных результатов: {len(partial_processed)} файлов")
            except:
                pass

        return processed_files

    def _create_batches(self, file_paths):
        """Создает батчи файлов на основе размера"""
        batches = []
        current_batch = []
        current_batch_size = 0

        for file_path in file_paths:
            file_size_gb = self._get_file_size_gb(file_path)

            # Если добавление файла превысит лимит батча, начинаем новый
            if current_batch_size + file_size_gb > self.batch_size and current_batch:
                batches.append(current_batch.copy())
                current_batch = []
                current_batch_size = 0

            # Добавляем файл в текущий батч
            current_batch.append(file_path)
            current_batch_size += file_size_gb

        # Добавляем последний батч
        if current_batch:
            batches.append(current_batch)

        # Оптимизация: стараемся сделать батчи более равномерными
        optimized_batches = self._optimize_batches(batches)

        return optimized_batches

    def _optimize_batches(self, batches):
        """Оптимизирует распределение файлов по батчам"""
        if len(batches) <= 1:
            return batches

        # Объединяем маленькие батчи
        optimized = []
        current_batch = []
        current_size = 0

        for batch in batches:
            batch_size = sum(self._get_file_size_gb(f) for f in batch)

            # Если текущий батч + этот батч <= лимита, объединяем
            if current_size + batch_size <= self.batch_size * 1.2:  # 20% допуск
                current_batch.extend(batch)
                current_size += batch_size
            else:
                if current_batch:
                    optimized.append(current_batch)
                current_batch = batch.copy()
                current_size = batch_size

        if current_batch:
            optimized.append(current_batch)

        print(f"🔧 Оптимизировано: {len(batches)} → {len(optimized)} батчей")

        return optimized

    def _process_batch(self, batch_files, batch_num):
        """Обрабатывает один батч файлов"""
        batch_results = []

        # 1. Очищаем локальные директории
        self._cleanup_local_dirs()

        # 2. Скачиваем файлы батча в локальную ФС
        local_files = []
        print(f"📥 Скачиваю {len(batch_files)} файлов в локальную ФС...")

        for file_path in batch_files:
            try:
                filename = os.path.basename(file_path)
                local_path = os.path.join(self.local_batch_dir, filename)

                # Копируем из Drive в локальную ФС
                shutil.copy2(file_path, local_path)
                local_files.append(local_path)

                # Проверяем, что файл скопировался
                if os.path.exists(local_path):
                    size_mb = os.path.getsize(local_path) / (1024*1024)
                    print(f"  ✅ {filename} ({size_mb:.1f} MB)")
                else:
                    print(f"  ❌ {filename} - не скопировался")

            except Exception as e:
                print(f"  ❌ Ошибка копирования {os.path.basename(file_path)}: {e}")

        print(f"📊 Скачано: {len(local_files)}/{len(batch_files)} файлов")

        if not local_files:
            print("⚠️  Нет файлов для обработки в этом батче")
            return []

        # 3. Обрабатываем каждый локальный файл
        print("\n🔊 Начинаю обработку аудио...")

        for i, local_file in enumerate(local_files, 1):
            try:
                print(f"\n[{i}/{len(local_files)}] Обрабатываю: {os.path.basename(local_file)}")

                # 3.1 Whisper: Аудио → Текст
                print("  📝 Шаг 1: Конвертация аудио в текст...")
                whisper_result = self._run_whisper_locally(local_file)

                if not whisper_result or 'text' not in whisper_result:
                    print("  ⚠️  Whisper не вернул текст")
                    continue

                text = whisper_result['text']
                print(f"  ✅ Текст извлечен ({len(text)} символов)")

                # 3.2 LLM: Текст → Теги
                print("  🏷️  Шаг 2: Тегирование текста...")
                tagging_result = self._run_tagging_locally(text)

                tags = tagging_result.get('result', [])
                summary = tagging_result.get('summary', '')

                print(f"  ✅ Добавлено тегов: {len(tags)}")

                # 3.3 Создаем результат
                result = {
                    'source_audio': os.path.basename(local_file),
                    'text': text,
                    'tags': str(tags),  # Сохраняем как строку
                    'summary': summary,
                    'text_length': len(text),
                    'processing_date': datetime.now().isoformat(),
                    'batch_number': batch_num,
                    'whisper_model': whisper_result.get('model', 'unknown'),
                    'audio_duration': whisper_result.get('duration', 0)
                }

                batch_results.append(result)

                # 3.4 Удаляем обработанный локальный файл
                os.remove(local_file)
                print(f"  🧹 Локальный файл удален")

            except Exception as e:
                print(f"  ❌ Ошибка обработки файла: {e}")
                import traceback
                traceback.print_exc()

        # 4. Очищаем временные файлы
        self._cleanup_local_dirs()

        print(f"\n✅ Батч {batch_num} обработан: {len(batch_results)}/{len(batch_files)} успешно")

        return batch_results


    def _run_whisper_locally(self, local_audio_path):
        """Запускает Whisper для локального аудиофайла"""


        try:
            return self.ap.process_file(local_audio_path, 7, False)
            # return {
            # 'date': date,
            # 'text': text,
            # 'source_file': os.path.basename(audio_path),
            # 'quality_score': quality_score,
            # 'tags': ''  # Пустая колонка для тегов
            # }

        except Exception as e:
            print(f"❌ Ошибка Whisper: {e}")
            return {'text': '', 'error': str(e)}


    def _run_tagging_locally(self, text):
        """Запускает тегирование для текста"""
        # ИНТЕГРАЦИЯ С ВАШИМ СКРИПТОМ ТЕГИРОВАНИЯ

        try:

          return self.tagger.get_tags_from_llm(text)
            # Пример вызова вашего скрипта
            # result = subprocess.run([
            #     'python', '/content/audio-processor/tagging_processor.py',
            #     '--text', text,
            #     '--model', 'mistral-nemo:12b'
            # ], capture_output=True, text=True)


        except Exception as e:
            print(f"❌ Ошибка тегирования: {e}")
            return {'result': [], 'summary': f'Ошибка: {str(e)}'}

    def _save_batch_results(self, batch_results, batch_num):
        """Сохраняет результаты батча"""
        if not batch_results:
            print("⚠️  Нет результатов для сохранения")
            return

        # Создаем DataFrame
        df = pd.DataFrame(batch_results)

        # Сохраняем в частичный файл
        partial_path = f"{self.partial_results_csv}.batch_{batch_num}"
        df.to_csv(partial_path, index=False, encoding='utf-8')

        # Добавляем в основной частичный файл
        if os.path.exists(self.partial_results_csv):
            existing_df = pd.read_csv(self.partial_results_csv)
            df = pd.concat([existing_df, df], ignore_index=True)

        if os.path.exists(self.partial_results_csv):
          df_existing = pd.read_csv(self.partial_results_csv)
          df = pd.concat([df_existing, df], ignore_index=True)
          df.to_csv(self.partial_results_csv, index=False, encoding='utf-8')

        print(f"💾 Результаты батча {batch_num} сохранены ({len(batch_results)} записей)")

    def _update_processed_list(self, processed_files):
        """Обновляет список обработанных файлов"""
        # Загружаем текущий список
        current_processed = self._load_processed_list()

        # Добавляем новые файлы
        new_processed = [os.path.basename(f) for f in processed_files]
        current_processed.update(new_processed)

        # Сохраняем
        with open(self.processed_files_log, 'w') as f:
            json.dump(list(current_processed), f)

        print(f"📝 Лог обновлен: +{len(new_processed)} файлов")

    def _consolidate_results(self):
        """Консолидирует все частичные результаты в один файл"""
        if not os.path.exists(self.partial_results_csv):
            print("⚠️  Нет частичных результатов для консолидации")
            return

        try:
            # Читаем все частичные результаты
            df = pd.read_csv(self.partial_results_csv)

            # Если уже есть основной файл, объединяем
            if os.path.exists(self.output_csv_path):
                existing_df = pd.read_csv(self.output_csv_path)
                df = pd.concat([existing_df, df], ignore_index=True)

            # Удаляем дубликаты
            df = df.drop_duplicates(subset=['source_audio'], keep='last')

            # Сохраняем
            df.to_csv(self.output_csv_path, index=False, encoding='utf-8')

            print(f"📦 Консолидировано {len(df)} записей в {self.output_csv_path}")

            # Очищаем временные файлы
            self._cleanup_temporary_files()

        except Exception as e:
            print(f"❌ Ошибка консолидации: {e}")

    def _cleanup_local_dirs(self):
        """Очищает локальные директории"""
        for dir_path in [self.local_temp_dir, self.local_whisper_dir, self.local_batch_dir]:
            if os.path.exists(dir_path):
                for file in os.listdir(dir_path):
                    try:
                        os.remove(os.path.join(dir_path, file))
                    except:
                        pass

        print("🧹 Локальные директории очищены")

    def _cleanup_temporary_files(self):
        """Очищает все временные файлы"""
        # Удаляем частичные результаты
        if os.path.exists(self.partial_results_csv):
            os.remove(self.partial_results_csv)

        # Удаляем файлы отдельных батчей
        for file in os.listdir("/content"):
            if file.startswith("partial_results") and file.endswith(".csv"):
                try:
                    os.remove(os.path.join("/content", file))
                except:
                    pass

        print("🧹 Временные файлы удалены")

    def get_processing_stats(self):
        """Возвращает статистику обработки"""
        stats = {
            'total_files': 0,
            'processed_files': 0,
            'remaining_files': 0,
            'progress_percent': 0,
            'estimated_size_gb': 0,
            'last_processed': None
        }

        try:
            # Получаем все файлы
            all_files = self._list_files_without_download()
            stats['total_files'] = len(all_files)
            stats['estimated_size_gb'] = self._estimate_total_size_gb(all_files)

            # Получаем обработанные файлы
            processed = self._load_processed_list()
            stats['processed_files'] = len(processed)
            stats['remaining_files'] = stats['total_files'] - stats['processed_files']

            if stats['total_files'] > 0:
                stats['progress_percent'] = (stats['processed_files'] / stats['total_files']) * 100

            # Получаем время последней обработки
            if os.path.exists(self.processed_files_log):
                mod_time = os.path.getmtime(self.processed_files_log)
                stats['last_processed'] = datetime.fromtimestamp(mod_time).strftime("%d.%m.%Y %H:%M")

        except Exception as e:
            print(f"⚠️  Ошибка получения статистики: {e}")

        return stats

# ============================================
# 🎯 ИСПОЛЬЗОВАНИЕ
# ============================================

def main():
    """Пример использования SmartAudioProcessor"""

    # Конфигурация
    DRIVE_AUDIO_PATH = "/content/drive/MyDrive/MCP_Call_Analytics/audio_raw"
    OUTPUT_CSV_PATH = "/content/drive/MyDrive/MCP_Call_Analytics/csv_calls/calls.csv"

    print("🤖 SMART AUDIO PROCESSOR - ЗАПУСК")
    print("="*50)

    # Инициализация процессора
    processor = SmartAudioProcessor(
        drive_audio_path=DRIVE_AUDIO_PATH,
        output_csv_path=OUTPUT_CSV_PATH,
        total_space_gb=50,      # Общее место в Colab
        batch_size_gb=2         # Батчи по 2 GB
    )

    # Показываем статистику
    stats = processor.get_processing_stats()
    print(f"\n📊 СТАТИСТИКА ДО ОБРАБОТКИ:")
    print(f"📁 Всего файлов: {stats['total_files']:,}")
    print(f"✅ Обработано: {stats['processed_files']:,}")
    print(f"🎯 Осталось: {stats['remaining_files']:,}")
    print(f"📈 Прогресс: {stats['progress_percent']:.1f}%")
    print(f"💾 Общий объем: {stats['estimated_size_gb']:.1f} GB")

    if stats['remaining_files'] == 0:
        print("\n🎉 Все файлы уже обработаны!")
        return

    # Запрашиваем подтверждение
    confirm = input(f"\n⚠️  Начать обработку {stats['remaining_files']:,} файлов? (y/n): ")

    if confirm.lower() != 'y':
        print("🚫 Обработка отменена")
        return

    # Запускаем обработку
    print("\n🚀 ЗАПУСК ОБРАБОТКИ...")
    results = processor.process_large_dataset()

    # Финальная статистика
    if results:
        print(f"\n🎉 ОБРАБОТКА ЗАВЕРШЕНА УСПЕШНО!")
        print(f"📊 Обработано записей: {len(results):,}")
        print(f"💾 Результаты: {OUTPUT_CSV_PATH}")
    else:
        print("\n⚠️  Обработка завершена без результатов")

main()


🤖 SMART AUDIO PROCESSOR - ЗАПУСК
Загрузка модели Whisper...


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 18.12 MiB is free. Process 2662 has 14.72 GiB memory in use. Of the allocated memory 14.06 GiB is allocated by PyTorch, and 545.78 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [11]:
import sys
sys.path.append('.')

from interactive import enhanced_interactive_mode

enhanced_interactive_mode(
    _model="mistral-nemo:12b",
    csv_dir=CSV_PATH,
    results_dir="/content/saved_results",
    drive_path=None
)


╔══════════════════════════════════════════╗
║    🤖 АНАЛИТИК ЗВОНКОВ v3.0               ║
║    🚀 Google Drive Edition                ║
╚══════════════════════════════════════════╝
    
✅ Ollama подключен, модель: mistral-nemo:12b
📂 Загружаю данные из JSON файлов...
📂 Читаю данные из CSV файла: calls.csv
✅ Загружено 202 строк из CSV
✅ Преобразовано 202 записей звонков
📅 Диапазон дат: 01.10.2025 - 20.01.2026
🏷️  Уникальных тегов: 19
✅ Загружено 202 звонков

[Local] 📝 Вопрос (/? для помощи): на что жаловались клиенты в октябре 2025
🔍 Анализирую: 'на что жаловались клиенты в октябре 2025'

🔍 Анализирую запрос: 'на что жаловались клиенты в октябре 2025'
🤖 Создаю план анализа...
   📅 Период: октябрь 2025 года
   🏷️  Теги: низкое_качество_стирки_или_чистки, не_заменили_ковры_вовремя, клиент_хочет_добавить_ковры, клиент_хочет_меньше_ковров, погашение_долга, расторжение_договора, возобновление_услуг, долго_нет_ответа_на_заявку, лишняя_доставка, доставили_не_те_ковры, не_выставлен_вовремя_счет,